# OCR libreria comprada

Este ocr requiere de la compra de licencia para la libreria https://ocr.space/ se require compra de licencia que nos entrega la key que es requerida archivos con múltiples columnas

se deben definir uno de los lenguajes [Optional]
Arabic=ara
Bulgarian=bul
Chinese(Simplified)=chs
Chinese(Traditional)=cht
Croatian = hrv
Czech = cze
Danish = dan
Dutch = dut
English = eng
Finnish = fin
French = fre
German = ger
Greek = gre
Hungarian = hun
Korean = kor
Italian = ita
Japanese = jpn
Polish = pol
Portuguese = por
Russian = rus
Slovenian = slv
Spanish = spa
Swedish = swe
Turkish = tur


insert into articulos_covid values('pais','fecha','medio','item1','item2',
'item3','tipo','nombre','txt0','txtc','txtt')

In [1]:
import ocrspace
import json
import requests
import os
from fnmatch import fnmatch
import pymssql

import time

In [32]:
idioma='eng'
pais_extraccion= 'UK'
apikey='OCRP8554898A'
pattern = "*.docx"
tipo_documento='Archivo PDF'

In [33]:


archivos_procesar=[]
arreglo=[val for sublist in [[os.path.join(i[0], j) for j in i[2]] for i in os.walk('../uk/')] for val in sublist]
# Meta comment to ease selecting text
for name in arreglo:
        if fnmatch(name, pattern):
            archivos_procesar.append(name)


print(len(arreglo))
print(len(archivos_procesar))

13450
1778


In [34]:
max=20
elmaslargo=''
for name in archivos_procesar:
    cantidad= len(archivos_procesar[100].split("/"))
    frase= archivos_procesar[100].split("/")
    if cantidad <max:
        max=cantidad
        elmaslargo=frase
print(max)
print(elmaslargo)

7
['..', 'uk', 'UK', 'JRP Press', '1Jan', 'The Guardian', 'P2Jan27.docx']


In [5]:
#api = ocrspace.API()
# Or if you have an API key or desired language, pass those:
#api = ocrspace.API(apikey, ocrspace.Language.Spanish)

In [35]:
def comprime_pdf(filename):
    arreglo=filename.split("/")
    pwd="/home/andres/"
    for i in range(len(arreglo)-2):
        pwd+=  arreglo[i+1]+"/"
    
    
    variable2=pwd+"archivo.pdf"
    fila=pwd+arreglo[-1]
    variable='gs -sDEVICE=pdfwrite -dCompatibilityLevel=1.6 -dPDFSETTINGS=/ebook -dNOPAUSE -dQUIET -dBATCH -sOutputFile="{}" "{}"'.format(variable2,fila)
    #print (variable)
    os.system(variable)
    return variable2, fila

In [36]:
def ocr_space_file(filename, overlay=False, api_key='holamund', language='eng'):
    """ OCR.space API request with local file.
        Python3.5 - not tested on 2.7
    :param filename: Your file path & name.
    :param overlay: Is OCR.space overlay required in your response.
                    Defaults to False.
    :param api_key: OCR.space API key.
                    Defaults to 'helloworld'.
    :param language: Language code to be used in OCR.
                    List of available language codes can be found on https://ocr.space/OCRAPI
                    Defaults to 'en'.
    :return: Result in JSON format.
    """

    payload = {'isOverlayRequired': overlay,
               'apikey': api_key,
               'language': language,
               'OCREngine':1
               }
    with open(filename, 'rb') as f:
        r = requests.post('https://apipro1.ocr.space/parse/image',
                          files={filename: f},
                          data=payload,
                          )
    return r.content.decode()

In [37]:
def conectar():
    conn = pymssql.connect("10.50.100.40", "aplicaciones", "disico", "prueba_arm_3")
    return conn

In [38]:
def limpia_linea(linea):
        linea = linea.replace("\n", " ")
        #linea = linea.replace("-", "")
        linea = linea.replace("\r", " ")
        return linea

In [39]:
def limpia_guion_espacio(linea):
        linea = linea.replace("-  ", "")
        #linea = linea.replace("-", "")
        return linea

In [40]:
print(archivos_procesar[0])

../uk/UK/JRP Press/3March/The Daily Mirror/P3Mar11COVID.docx


In [41]:
tmp_arreglo= archivos_procesar

In [42]:
print(len(tmp_arreglo))

archivos_procesar=tmp_arreglo[0:]
print(len(archivos_procesar))

1778
1778


In [ ]:
retorno= ocr_space_file(archivos_procesar[0],False, apikey,idioma)
jsoon_data=json.loads(retorno)
print(jsoon_data)
texto = jsoon_data['ParsedResults'][0]['ParsedText']
#texto2=limpia_linea(texto)

In [15]:
variable2, fila=comprime_pdf(archivos_procesar[0])
#print("/home/andresramos/ocr/chile/11 abril/biobio.cl/23 PM/Noticias/L790/L790.pdf".split("/"))
print(fila.split("/"))
print(variable2)

['', 'home', 'andres', 'uk', 'UK', 'JRP Press', '3March', 'The Guardian', 'P2Mar36.pdf']
/home/andres/uk/UK/JRP Press/3March/The Guardian/archivo.pdf


In [ ]:
### PROCESAR PDFS #####

count=0
for fila2 in archivos_procesar:
        fila2=fila2.replace("’", "")
        variable2, fila=comprime_pdf(fila2)
        retorno= ocr_space_file(variable2,False, apikey,idioma)
        jsoon_data=json.loads(retorno)
        texto = jsoon_data['ParsedResults'][0]['ParsedText']
        texto2=limpia_linea(texto)
        frase_fila= fila.split("/")
        pais= pais_extraccion
        fecha= frase_fila[6]
        medio= frase_fila[7]
        if len(frase_fila)>=9:
                item1= frase_fila[6]
        else:
            item1= 'otros'
        if len(frase_fila)>=10:
                item2= frase_fila[8]
        else:
            item2= 'otros'
        item3= 'otros'
        tipo=tipo_documento 
        nombre=fila.split("/")[-1].split(".")[0]
        txt0=limpia_linea(texto)
        txtc= limpia_guion_espacio(txt0)
        txtt= 'pendiente'
        conn= conectar()
        cursor2 = conn.cursor()
        cursor2.executemany(
                        "INSERT INTO articulos_covid VALUES ( %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)",
                        [(pais,fecha,medio,item1,item2, item3,tipo,nombre,txt0,txtc,txtt)])
                        # you must call commit() to persist your data if you don't set autocommit to True
        conn.commit()
        cursor2.close()
        conn.close()

        if count%55==0:
            print(count)
            localtime = time.localtime()
            print(time.strftime("%I:%M:%S %p", localtime))
            time.sleep(601)
            localtime = time.localtime()
            print(time.strftime("%I:%M:%S %p", localtime))
        count+=1
  
    

In [77]:
def docx2pdf (filename):
    lugar_almacenamiento ="/home/andres/docx2pdf_temp"
    filename = filename.replace(' ', '\ ')
    arreglo=filename.split("/")
    pwd="/home/andres/"
    for i in range(len(arreglo)-2):
        pwd+=  arreglo[i+1]+"/"
    print(filename)
    print(pwd)
    comando = 'lowriter --convert-to pdf --outdir {} {}'.format(lugar_almacenamiento,filename)
    os.system(comando)
    archivo= filename.split("/")
    nom_archivo= archivo[-1].split(".")
    print (nom_archivo[0])
    url_archivo= lugar_almacenamiento+"/"+nom_archivo[-2]+".pdf"
    print(url_archivo)
    return url_archivo, nom_archivo[0]
    

In [76]:
docx2pdf(archivos_procesar[0])

../uk/UK/JRP\ Press/3March/The\ Daily\ Mirror/P3Mar11COVID.docx
/home/andres/uk/UK/JRP\ Press/3March/The\ Daily\ Mirror/P3Mar11COVID.docx/
P3Mar11COVID
/home/andres/docx2pdf_temp/P3Mar11COVID.pdf


('/home/andres/docx2pdf_temp/P3Mar11COVID.pdf', 'P3Mar11COVID')

In [ ]:
### PROCESAR DOCX #####

count=0
for fila2 in archivos_procesar:
        fila2=fila2.replace("’", "")
        fila2, nombre_archivo=docx2pdf(fila2)
        variable2, fila_no=comprime_pdf(fila2)
        retorno= ocr_space_file(variable2,False, apikey,idioma)
        jsoon_data=json.loads(retorno)
        texto = jsoon_data['ParsedResults'][0]['ParsedText']
        texto2=limpia_linea(texto)
        frase_fila= fila.split("/")
        pais= pais_extraccion
        fecha= frase_fila[6]
        medio= frase_fila[7]
        if len(frase_fila)>=9:
                item1= frase_fila[6]
        else:
            item1= 'otros'
        if len(frase_fila)>=10:
                item2= frase_fila[8]
        else:
            item2= 'otros'
        item3= 'otros'
        tipo=tipo_documento 
        nombre=fila.split("/")[-1].split(".")[0]
        txt0=limpia_linea(texto)
        txtc= limpia_guion_espacio(txt0)
        txtt= 'pendiente'
        conn= conectar()
        cursor2 = conn.cursor()
        cursor2.executemany(
                        "INSERT INTO articulos_covid VALUES ( %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)",
                        [(pais,fecha,medio,item1,item2, item3,tipo,nombre,txt0,txtc,txtt)])
                        # you must call commit() to persist your data if you don't set autocommit to True
        conn.commit()
        cursor2.close()
        conn.close()

        if count%55==0:
            print(count)
            localtime = time.localtime()
            print(time.strftime("%I:%M:%S %p", localtime))
            time.sleep(601)
            localtime = time.localtime()
            print(time.strftime("%I:%M:%S %p", localtime))
        count+=1
  

In [ ]:
texto3=limpia_guion_espacio(texto2)

In [ ]:
print (texto3)